In [ ]:
import pandas as pd

from util import Pipeline

p = Pipeline('configs')

In [ ]:
base_year = p.settings['base_year']

In [109]:
df = p.get_table('ofm_estimates_2020_by_control_area')

In [111]:
df

,control_id,ofm_units,ofm_hh,ofm_gq,ofm_hhpop,ofm_total_pop
0,1,25653.389015,24823.992023,1011.000000,63433.102854,64444.102854
1,2,171052.697040,161487.045974,7951.874976,361567.847057,369519.722033
2,3,15209.444969,14375.112970,382.000000,38902.717894,39284.717894
3,4,8082.601011,7701.966983,76.000000,18862.424107,18938.424107
4,5,10398.697010,9966.016017,260.000000,27291.153029,27551.153029
...,...,...,...,...,...,...
149,304,6917.268024,6199.146985,34.000000,17504.874978,17538.874978
150,401,16.519000,16.246000,969.242000,46.193999,1015.435999
151,402,780.474006,734.398016,948.325012,2414.855987,3363.180999
152,403,2053.699982,1877.110994,3147.003981,6556.141071,9703.145051


,target_id,start,horizon,unit_chg,unit_chg_adj
0,1,2019,2044,35000,34580
1,2,2019,2044,112000,109453
2,3,2019,2044,12000,11996
3,4,2019,2044,5800,5457
4,5,2019,2044,7500,7428
5,6,2019,2044,11260,11216
6,7,2019,2044,3500,3094
7,8,2019,2044,10200,10065
8,9,2019,2044,13200,12869
9,10,2019,2044,20000,19826
